<h1>Actividad SPARK ML - Sistema de recomendación</h1>

<h2>Importar librerías necesarias</h2>

In [1]:
import findspark
import pandas as pd
import pyspark
import sys
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType
from IPython.core.display import display, HTML
from urllib.request import urlopen
import json
import time

pd.set_option('display.max_rows', None)


spark = SparkSession.builder.getOrCreate()

/tmp/ipykernel_8963/895906536.py:14: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/18 13:18:19 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/12/18 13:18:19 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/12/18 13:18:19 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/12/18 13:18:20 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


<h2>Leer CSV</h2>

In [2]:
df_anime= spark.read.csv('gs://bd_anime/anime.csv', header=True, inferSchema=True)
df_ratings_complete= spark.read.csv('gs://bd_anime/rating_complete.csv', header=True, inferSchema=True)
column_names = df_ratings_complete.columns
df_valoraciones_ep= spark.read.csv('gs://bd_anime/valoraciones_EP.csv', inferSchema=True).toDF(column_names[0],column_names[1],column_names[2])

<h2>Ejercicio1</h2>

Hacer un análisis exploratorio de los datos con pySpark, mostrando información relevante de los mismos (cuáles son los ítems mejor y peor valorados, la relación entre género y valoraciones, estudios con mejor y peor nota media, etc...).

In [3]:
#Vemos todas las columnas que tiene nuestro csv de anime
df_anime.columns

['ID',
 'Name',
 'Score',
 'Genres',
 'English name',
 'Japanese name',
 'Type',
 'Episodes',
 'Aired',
 'Premiered',
 'Producers',
 'Licensors',
 'Studios',
 'Source',
 'Duration',
 'Rating',
 'Ranked',
 'Popularity',
 'Members',
 'Favorites',
 'Watching',
 'Completed',
 'On-Hold',
 'Dropped',
 'Plan to Watch',
 'Score-10',
 'Score-9',
 'Score-8',
 'Score-7',
 'Score-6',
 'Score-5',
 'Score-4',
 'Score-3',
 'Score-2',
 'Score-1']

In [4]:
#Borramos todas las columnas menos las que nos interesan
df_limpio = df_anime.drop(
#'ID',
#'Name',
#'Score',
#'Genres',
#'English name',
 'Japanese name',
#'Type',
#'Episodes',
 'Aired',
 'Premiered',
 'Producers',
 'Licensors',
#'Studios',
 'Source',
 'Duration',
 'Rating',
 'Ranked',
 'Popularity',
 'Members',
 'Favorites',
 'Watching',
 'Completed',
 'On-Hold',
 'Dropped',
 'Plan to Watch',
 'Score-10',
 'Score-9',
 'Score-8',
 'Score-7',
 'Score-6',
 'Score-5',
 'Score-4',
 'Score-3',
 'Score-2',
 'Score-1'
)

<h3>ITEM MEJOR VALORADO</h3>

In [5]:
df_valitem = df_limpio
df_valitem = df_valitem.filter("Score >= 0 ")
df_valitem=df_valitem.orderBy('Score',ascending=False)
df_valitem.show(1)

+----+--------------------+-----+--------------------+--------------------+----+--------+-------+
|  ID|                Name|Score|              Genres|        English name|Type|Episodes|Studios|
+----+--------------------+-----+--------------------+--------------------+----+--------+-------+
|5114|Fullmetal Alchemi...| 9.19|Action, Military,...|Fullmetal Alchemi...|  TV|      64|  Bones|
+----+--------------------+-----+--------------------+--------------------+----+--------+-------+
only showing top 1 row



<h3>ITEM PEOR VALORADO</h3>

In [6]:
df_valitem=df_valitem.orderBy('Score',ascending=True)
df_valitem.show(1)

+----+--------------------+-----+-------------+------------+----+--------+-------+
|  ID|                Name|Score|       Genres|English name|Type|Episodes|Studios|
+----+--------------------+-----+-------------+------------+----+--------+-------+
|3287|Tenkuu Danzai Ske...| 1.85|Sci-Fi, Mecha|     Unknown| OVA|       1|Unknown|
+----+--------------------+-----+-------------+------------+----+--------+-------+
only showing top 1 row



<h3>Valoracion media de cada genero</h3>

In [7]:
df_generos = df_limpio
df_generos = df_generos.select(split(col('Genres'),','),col('Score')).withColumnRenamed('split(Genres, ,, -1)','Genres')
df_generos = df_generos.select(explode(col('Genres')),col('Score')).withColumnRenamed('col','Genres')
df_generos = df_generos.withColumn('Genres', trim(col('Genres')))
df_generos = df_generos.groupBy('Genres').agg(F.mean('Score')).withColumnRenamed('avg(Score)','Score').orderBy('Score',ascending=False)

df_generos.show(1000)

+-------------+------------------+
|       Genres|             Score|
+-------------+------------------+
|     Thriller| 7.123508771929825|
|      Mystery| 7.108549141965681|
|      Shounen|  7.03356347438752|
|       Police| 7.010000000000004|
|        Drama| 6.977232796486086|
|        Josei|6.9674725274725295|
|       Seinen| 6.966082758620689|
|Psychological|6.9140880503144695|
| Supernatural| 6.909076923076922|
|       School| 6.900921144452192|
|  Super Power| 6.884165067178506|
|      Romance|6.8837703016241205|
|     Military| 6.871941176470591|
|       Sports| 6.846988847583642|
|Slice of Life| 6.838810218978104|
|      Samurai| 6.826951219512197|
|       Shoujo|  6.81998366013072|
|      Vampire|6.8179230769230745|
|    Shoujo Ai| 6.817792207792206|
|   Historical|6.8094490358126745|
|        Harem| 6.773033419023142|
| Martial Arts|6.7717559523809525|
|        Magic| 6.761361655773423|
|    Adventure| 6.755227272727276|
|       Action| 6.750751811594207|
|   Shounen Ai| 6.72

<h3>Estudios con mejor nota media</h3>

In [8]:
df_estudio = df_limpio
df_estudio = df_estudio.select(split(col('Studios'),','),col('Score')).withColumnRenamed('split(Studios, ,, -1)','Studios')
df_estudio = df_estudio.select(explode(col('Studios')),col('Score')).withColumnRenamed('col','Studios')
df_estudio = df_estudio.withColumn('Studios', trim(col('Studios')))
df_estudio = df_estudio.groupBy('Studios').agg(F.mean('Score')).withColumnRenamed('avg(Score)','Score').orderBy('Score',ascending=False)
df_estudio.show()

+--------------------+-----------------+
|             Studios|            Score|
+--------------------+-----------------+
|         Studio Bind|             8.37|
|            Egg Firm|           8.3225|
|        Studio Chizu|8.096666666666666|
|Nippon Ramayana F...|             8.04|
|Colored-Pencil An...|             8.03|
|Samsara Animation...|              8.0|
|         Djinn Power|             7.83|
|Purple Cow Studio...|7.720000000000001|
|                Code|             7.71|
|               Shuka|7.705625000000001|
| Nakamura Production|              7.7|
|           Odolttogi|             7.62|
|             Fuji TV|              7.6|
|         Twin Engine|             7.57|
|Tencent Animation...|            7.555|
|        Animation Do|7.553333333333334|
|       Studio DURIAN|             7.52|
| Wan Wei Mao Donghua|              7.5|
|      Original Force|              7.5|
|             BeSTACK|            7.495|
+--------------------+-----------------+
only showing top

<h2>Ejercicio 2</h2>
Crear un programa con Spark ML y el algoritmo de recomendación ALS que genere un listado con 5 serie de TV y 5 películas para recomendar al usuario EP (con id 666666) para ello se deben incorporar las valoraciones de EP al fichero de valoraciones total, entrenar el algoritmo y pedirle un listado de recomendaciones para ese usuario. Las recomendaciones generadas deben incluir el ID del anime y los títulos original (name) y en inglés (English name). El listado debe aparecer ordenado por valoración media de cada serie/película. 

In [9]:
#Uno el CSV de Ratings con el del usuario EP
df_valoraciones = df_ratings_complete.union(df_valoraciones_ep)
df_valoraciones.count()

57633343

In [10]:
#Uno el dataframe de valoraciones con el CSV de anime
df_final = df_valoraciones.join(df_limpio, df_valoraciones.anime_id == df_limpio.ID).drop(col('ID'))
df_final = df_final.where(col('Score') >= 0)
df_final.show(10)


+-------+--------+------+--------------------+-----+--------------------+--------------------+-----+--------+----------------+
|user_id|anime_id|rating|                Name|Score|              Genres|        English name| Type|Episodes|         Studios|
+-------+--------+------+--------------------+-----+--------------------+--------------------+-----+--------+----------------+
|      0|     430|   9.0|Fullmetal Alchemi...| 7.57|Military, Comedy,...|Fullmetal Alchemi...|Movie|       1|           Bones|
|      0|    1004|   5.0|Kanojo to Kanojo ...| 7.33|Drama, Psychologi...|She and Her Cat:T...|  OVA|       1|         Unknown|
|      0|    3010|   7.0|      Kaiketsu Zorro| 7.23|Adventure, Histor...|The Magnificent Z...|   TV|      52| Ashi Production|
|      0|     570|   7.0|             Jin-Rou| 7.79|Military, Police,...|Jin-Roh:The Wolf ...|Movie|       1|  Production I.G|
|      0|    2762|   9.0|      Igano Kabamaru| 7.87|Action, Adventure...|             Unknown|   TV|      24|  

In [11]:
#Creo el dataframe de peliculas
df_peliculas = df_final.filter( (col('Type')=='Movie') | ((col('Type')=='OVA') & (col('Episodes')=='1')) )
df_peliculas.show(10)
#Creo el dataframe de series de TV
df_seriestv = df_final.filter( col('Type')=='TV' )
df_seriestv.show(10)

+-------+--------+------+--------------------+-----+--------------------+--------------------+-----+--------+----------------+
|user_id|anime_id|rating|                Name|Score|              Genres|        English name| Type|Episodes|         Studios|
+-------+--------+------+--------------------+-----+--------------------+--------------------+-----+--------+----------------+
|      0|     430|   9.0|Fullmetal Alchemi...| 7.57|Military, Comedy,...|Fullmetal Alchemi...|Movie|       1|           Bones|
|      0|    1004|   5.0|Kanojo to Kanojo ...| 7.33|Drama, Psychologi...|She and Her Cat:T...|  OVA|       1|         Unknown|
|      0|     570|   7.0|             Jin-Rou| 7.79|Military, Police,...|Jin-Roh:The Wolf ...|Movie|       1|  Production I.G|
|      0|     431|   8.0| Howl no Ugoku Shiro| 8.67|Adventure, Drama,...|Howl's Moving Castle|Movie|       1|   Studio Ghibli|
|      0|     578|  10.0|      Hotaru no Haka| 8.51|   Drama, Historical|Grave of the Fire...|Movie|       1|  

In [12]:
#Entrenamos el sistema de recomenadición de películas
(trainingP, testP) = df_peliculas.randomSplit([0.8,0.2])
als = ALS(maxIter=5, regParam=0.01, userCol='user_id', itemCol='anime_id', ratingCol='rating', coldStartStrategy='drop')
peliculas = als.fit(trainingP)
predicciones_peliculas = peliculas.transform(testP)
#Entrenamos el sistema de recomenadición de series
(trainingS, testS) = df_seriestv.randomSplit([0.8,0.2])
als = ALS(maxIter=5, regParam=0.01, userCol='user_id', itemCol='anime_id', ratingCol='rating', coldStartStrategy='drop')
series = als.fit(trainingS)
predicciones_series = series.transform(testS)

In [19]:
#Generamos la recomendación de 5 peliculas para el usuario elegido
users = df_valoraciones_ep.select(als.getUserCol()).distinct().limit(1)
peliculasRecomendadas = peliculas.recommendForUserSubset(users, 5).head()
recomendaciones = peliculasRecomendadas[1]
idsPeliculas = []
for recomendacion in recomendaciones:
    idsPeliculas.append(recomendacion[0])

df_topPelis = df_limpio.filter(col('ID').isin(idsPeliculas)).orderBy('Score',ascending=False).drop('Type', 'Episodes', 'Studios')
print('Peliculas')
df_topPelis.show()

#Generamos la recomendación de 5 series para el usuario elegido
seriesRecomendadas = series.recommendForUserSubset(users, 5).head()
recomendaciones = seriesRecomendadas[1]
idsSeries = []
for recomendacion in recomendaciones:
    idsSeries.append(recomendacion[0])

df_topSeries = df_limpio.filter(col('ID').isin(idsSeries)).orderBy('Score',ascending=False).drop('Type', 'Episodes', 'Studios')
print('Series')
df_topSeries.show()
df_topPelis.toPandas().to_csv('gs://bd_anime/recomendacionesPelis.txt')
df_topSeries.toPandas().to_csv('gs://bd_anime/recomendacionesSeries.txt')

Peliculas
+-----+--------------------+-----+--------------------+--------------------+
|   ID|                Name|Score|              Genres|        English name|
+-----+--------------------+-----+--------------------+--------------------+
|33238|         Padak Padak| 6.66|               Drama|               PADAK|
|26111|     Hibike! Wadaiko| 6.63|         Drama, Kids|             Unknown|
|30481|Ginga Tetsudou 99...| 6.38|Adventure, Space,...|             Unknown|
|30193|           Eun-sil-i| 5.95|               Drama|         The Dearest|
| 8081|Je Bul Chal-ssi I...| 5.81|Comedy, Horror, F...|The Story of Mr. ...|
+-----+--------------------+-----+--------------------+--------------------+



Series
+-----+--------------------+-----+--------------------+--------------------+
|   ID|                Name|Score|              Genres|        English name|
+-----+--------------------+-----+--------------------+--------------------+
| 9253|         Steins;Gate| 9.11|    Thriller, Sci-Fi|         Steins;Gate|
|28977|            Gintama°|  9.1|Action, Comedy, H...|    Gintama Season 4|
|15417| Gintama': Enchousen| 9.04|Action, Comedy, H...|   Gintama:Enchousen|
|34096|            Gintama.| 8.99|Action, Comedy, H...|    Gintama Season 5|
|37491|Gintama.: Shiroga...| 8.86|Action, Sci-Fi, C...|Gintama.:Silver S...|
+-----+--------------------+-----+--------------------+--------------------+



<h3>Ejercicio 3</h3>
Además se debe recuperar y mostrar información detallada (sinopsis, imagen, trailer, etc.) de cada serie o película extrayendo la información con la API Jikan (https://docs.api.jikan.moe Links to an external site.). Por ejemplo, para recuperar la información básica del anime con ID 13601, hay que hacer una llamada a la URL https://api.jikan.moe/v4/anime/13601/full Links to an external site.y luego parsear los datos del JSON que devuelve. Si la práctica se hace usando un cuaderno de Jupyter, mostrar la información recuperada de forma gráfica en el cuaderno. Si en vez de cuaderno se ha creado un archivo .py que se ejecuta desde el terminal, mostrar información usando la librería "rich" de Python (https://github.com/Textualize/rich Links to an external site.).

In [20]:
def recorrerItems(ids):
    for id in ids:
        url = "https://api.jikan.moe/v4/anime/"+str(id)+"/full"
        response = urlopen(url)
        leerInfo(json.loads(response.read()).get('data'))

def leerInfo(data):
    time.sleep(0.5)
    display(HTML('<h2>'+data.get('title')+'</h2>'))
    display(HTML('<img src="'+data.get('images').get('jpg').get('image_url')+'">'))
    try:
        display(HTML('<iframe width="420" height="315" src="'+data.get('trailer').get('embed_url')+'"></iframe>'))
        display(HTML('<p><b>Synopsis: </b>'+ data.get('synopsis')+'</p>'))
        display(HTML('<p><b>Score: </b>'+ str(data.get('score'))+'</p>'))
        display(HTML('<p><b>Year: </b>'+ str(data.get('year'))+'</p>'))
        display(HTML('<p><b>Episodes: </b>'+ str(data.get('episodes')) +'</p>')) 
    except:
        display(HTML('<p><b>Synopsis: </b>'+ data.get('synopsis')+'</p>'))
        display(HTML('<p><b>Score: </b>'+ str(data.get('score'))+'</p>'))
        display(HTML('<p><b>Year: </b>'+ str(data.get('year'))+'</p>'))
        display(HTML('<p><b>Episodes: </b>'+ str(data.get('episodes')) +'</p>'))        

display(HTML('<h1>Recomendaciones de peliculas</h1>'))
recorrerItems(idsPeliculas)
display(HTML('<h1>Recomendaciones de series</h1>'))
recorrerItems(idsSeries)   

/opt/conda/miniconda3/lib/python3.8/site-packages/IPython/core/display.py:431: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")
